# import and install dependencies

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import tensorflow as tf


print("TensorFlow version:", tf.__version__)
print("OpenCV version:", cv2.__version__)

# keypoits using mp holistic

In [ ]:
mp_holistic = mp.solutions.holistic  # Holistic model
mp_drawing = mp.solutions.drawing_utils  # Drawing utilities

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False                 
    results = model.process(image)                 
    image.flags.writeable = True                  
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

In [ ]:
def draw_landmarks(image, results):
    #  face connections 
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) 
    #  pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) 
    #  left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 
    #  right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [ ]:
def draw_styled_landmarks(image, results):
  
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(224, 224, 224), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=1, circle_radius=1)  
                             )
    

    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(57, 255, 20), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(0, 128, 0), thickness=2, circle_radius=2)    
                             )
    
    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(255, 255, 0), thickness=2, circle_radius=4),  
                             mp_drawing.DrawingSpec(color=(128, 128, 0), thickness=2, circle_radius=2)   
                             )

    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(255, 0, 255), thickness=2, circle_radius=4),  
                             mp_drawing.DrawingSpec(color=(128, 0, 128), thickness=2, circle_radius=2)  
                             )

In [ ]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        
        image, results = mediapipe_detection(frame, holistic)
        
    
        draw_styled_landmarks(image, results)


        cv2.imshow('OpenCV Feed', image)

        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
len(results.left_hand_landmarks.landmark)

In [ ]:
results

In [ ]:

draw_styled_landmarks(frame, results)


plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
plt.show()

# extract folders for collections

In [ ]:
len(results.pose_landmarks.landmark)

In [ ]:
def extract_keypoints(results):
    # 1. Pose: 33 landmarks * 4 values (x, y, z, visibility) = 132 values
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    
    # 2. Face: 468 landmarks * 3 values (x, y, z) = 1404 values
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    
    # 3. Left Hand: 21 landmarks * 3 values (x, y, z) = 63 values
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    
    # 4. Right Hand: 21 landmarks * 3 values (x, y, z) = 63 values
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    
    # 5. Concatenate everything into one big array (Total: 1662 values)
    return np.concatenate ([pose,face,lh,rh])

In [ ]:
result_test = extract_keypoints(results)

In [ ]:
result_test

In [ ]:
np.save('0',result_test)

In [ ]:
np.load('0.npy')

# setup folders for collections

In [ ]:
import os
import numpy as np


actions = np.load("actions.npy")


DATA_PATH = os.path.join('MP_Data')


no_sequences = 30
sequence_length = 30


for action in actions:
    for sequence in range(no_sequences):
        os.makedirs(os.path.join(DATA_PATH, action, str(sequence)), exist_ok=True)

print("Folders verified using frozen actions:", actions)


# collect keypoints values for training and testing 

In [ ]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    stop_recording = False 
    
    for action in actions:
        if stop_recording: break 
            
        for sequence in range(no_sequences):
            if stop_recording: break 
                
            for frame_num in range(sequence_length):
                ret, frame = cap.read()
                image, results = mediapipe_detection(frame, holistic)
                draw_styled_landmarks(image, results)
                
                
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, f'Collecting {action} Video {sequence}', (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000) 
                else: 
                    cv2.putText(image, f'Collecting {action} Video {sequence}', (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)

                
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    stop_recording = True
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

# preprocess data and create lables and features

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label : num for num , label in enumerate(actions)}

In [ ]:
label_map

In [ ]:
#                          this code is for loading actions so that we do not have to recapture the frames again

In [ ]:
sequences,labels = [],[]                                 
for action in actions:
    for sequence in range(no_sequences):
     window = []
     for frame_num in range(sequence_length):
        res = np.load(os.path.join(DATA_PATH , action , str(sequence), "{}.npy".format(frame_num)))
        window.append(res)
     sequences.append(window)
     labels.append(label_map[action])

In [ ]:
np.array(sequences).shape

In [ ]:
np.array(labels).shape

In [ ]:
x = np.array(sequences)

In [ ]:
x.shape

In [ ]:
y = to_categorical(labels, num_classes=len(actions))


In [ ]:
y

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y,
    test_size=0.1,
    random_state=42,
    stratify=y.argmax(axis=1)
)


In [ ]:
y_test.shape

# Build and  Train LSTM Neural Network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir = log_dir)  # use to monitor our neural network train

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences = True,activation = 'relu',input_shape = (30,1662)))
model.add(LSTM(128, return_sequences = True,activation = 'relu'))
model.add(LSTM(64, return_sequences = False,activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(actions.shape[0],activation = 'softmax'))

In [ ]:
res = [.7,0.2,0.1]


In [ ]:
actions[np.argmax(res)]

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs=2000, callbacks = [tb_callback])

In [ ]:
# Save the model so you don't have to train it again
model.save('action.h5')
print(" Model saved as action.h5!")

In [ ]:
print(model.metrics_names)


In [ ]:
model.summary()

# make predictions

In [ ]:
res = model.predict(x_test)

In [ ]:
actions[np.argmax(res[0])]

In [ ]:
actions[np.argmax(y_test[0])]

# Evaluation using Confussion matrix and accuracy 

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score

In [ ]:
yhat = model.predict(x_test)

In [ ]:
ytrue = np.argmax(y_test,axis = 1).tolist()
yhat = np.argmax(yhat,axis = 1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue,yhat)

In [ ]:
accuracy_score(ytrue,yhat)

# test in real time 

In [ ]:
# We need 6 colors for 6 signs
colors = [(245,117,16), (117,245,16), (16,117,245), (255,0,0), (0,255,0), (0,0,255)]

def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
    
        cv2.rectangle(output_frame, (0, 60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:

if len(res.shape) > 1:
    res = res[0]


if results.left_hand_landmarks or results.right_hand_landmarks:
   
    processed_image = prob_viz(res, actions, image, colors)
    plt.imshow(processed_image)
else:
    print("Error: No hands detected in the current frame to visualize!")
    plt.imshow(image) 

In [ ]:

sequence = []
sentence = []
predictions = []
threshold = 0.6  

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)
        
        
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:] 
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))
            
            
            if len(predictions) >= 5:
                if np.unique(predictions[-5:])[0] == np.argmax(res): 
                    if res[np.argmax(res)] > threshold: 
                        if len(sentence) > 0: 
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])
            

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        cv2.imshow('Fast Detection - 6 Signs', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
model.predict(x_test)